**Progetto Machine Learning**
---
**Giovanni D'Agostino, Marco Giorgi**

Fase Preliminare: 
- individuare la GPU Tesla-P100
- montare il drive contenente il codice e i dataset

In [ ]:
########## Tipo GPU allocata #############
!nvidia-smi

Sat Aug 28 10:31:59 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 470.57.02    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla P4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   36C    P8     7W /  75W |      0MiB /  7611MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
from google.colab import drive
import sys
drive.mount('Project_ML')
sys.path.append('/content/Project_ML/My Drive/Tesi/ProveFinali/Fridge')

Mounted at Project_ML


# **Load Dataset**



Fase di Caricamento:
- import librerie
- load dei dataset main e fridge
- descrizione base dei dataset


In [ ]:
import preprocessing 
import pandas as pd
import model
import metrics
import generator
import stats_data
import tensorflow as tf
import numpy as np

ModuleNotFoundError: ignored

In [ ]:
pd.set_option('float_format', '{:f}'.format)
df_main = pd.read_csv("/content/Project_ML/My Drive/ML/ProveFinali/main_train.csv")
df_main.describe(include='all')

In [ ]:
df_fridge = pd.read_csv("/content/Project_ML/My Drive/ML/ProveFinali/fridge_train.csv")
df_fridge.describe(include='all')

# **Sperimentazione**

Fase di sperimentazione:
- configurazione size di window e batch
- preprocessing (padding + standardizzazione)
- statistiche basi (media + deviazione standard)
- modello
- configurazione generatore
- configurazione early stopping
- fit
- plot risultati

In [ ]:
window_size= 3999 # NB!deve avere dimensione dispari
batch_size = 512

In [ ]:
main_power = preprocessing.load_data(df_main)
train_main, val_main = preprocessing.split_data(main_power)
mean_main=np.mean(train_main) #### media main train
std_main= np.std(train_main)  ##### dev standard main train
train_main_pad = preprocessing.padding(train_main, window_size)
val_main_pad = preprocessing.padding(val_main, window_size)
train_main_standard = preprocessing.standardizzazione(train_main_pad, mean_main, std_main)
val_main_standard = preprocessing.standardizzazione(val_main_pad, mean_main, std_main)

fridge_power= preprocessing.load_data(df_fridge)
train_f, val_f = preprocessing.split_data(fridge_power)
mean_fridge=np.mean(train_f) ##### mean fridge train
std_fridge=np.std(train_f) ##### dev std fridge train
train_f_pad= preprocessing.padding(train_f, window_size)
val_f_pad= preprocessing.padding(val_f, window_size)
train_fridge_standard = preprocessing.standardizzazione(train_f_pad, mean_fridge, std_fridge)
val_fridge_standard = preprocessing.standardizzazione(val_f_pad, mean_fridge, std_fridge)


In [ ]:
preprocessing.stat("Main_Train", "Fridge_Train",train_main, train_f)
preprocessing.stat("Main_Val", "Fridge_Val",val_main, val_f)

In [ ]:
model_fridge = model.fridge_model(window_size, metrics.F1_score(scaling='strd', Mean=mean_fridge, Std=std_fridge))
model_fridge.summary()

In [ ]:
train_generator = generator.DataGenerator(train_main_standard, train_fridge_standard, window_len=window_size, batch_len=batch_size)
val_generator = generator.DataGenerator(val_main_standard, val_fridge_standard, window_len=window_size, batch_len=batch_size, shuffle=False)

In [ ]:
early_stopping = tf.keras.callbacks.EarlyStopping(monitor='val_f1_score', mode='max', patience=5)
callbacks = [early_stopping]

In [ ]:
fit_data = model_fridge.fit(x=train_generator, epochs=10, steps_per_epoch= train_generator.__len__(), validation_data = val_generator, validation_steps= val_generator.__len__(), callbacks= callbacks )
stats_data.plot_fit(fit_data)